In [16]:
# Import necessary libraries
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer


In [17]:
# Step 1: Load the Dataset
ds = load_dataset("uitnlp/vietnamese_students_feedback")
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [18]:
# create train_dataset and eval_dataset
train_dataset = ds['train']
eval_dataset = ds['validation']


In [22]:
# Step 2: Load a Pre-trained Tokenizer (Using PhoBERT)
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
# Step 3: Define a Tokenization Function
def preprocess_function(examples):
    # Tokenize the 'sentence' field
    tokenized = tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)
    tokenized['label'] = examples['sentiment']
    return tokenized

/home/hoang.minh.an/miniconda3/envs/llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
# Step 4: Apply Tokenization to Each Split of the Dataset
tokenized_ds = ds.map(preprocess_function, batched=True)

# Step 5: Prepare Training, Validation, and Test Datasets
train_ds = tokenized_ds['train']
validation_ds = tokenized_ds['validation']
test_ds = tokenized_ds['test']

# Step 6: Convert Labels to Numerical Values (using 'sentiment')
if not isinstance(train_ds.features['sentiment'], ClassLabel):
    # Convert to ClassLabel for automatic mapping
    train_ds = train_ds.class_encode_column('sentiment')
    validation_ds = validation_ds.class_encode_column('sentiment')
    test_ds = test_ds.class_encode_column('sentiment')

# Get the number of unique labels (sentiments)
num_labels = train_ds.features['sentiment'].num_classes

# Step 7: Create a Data Collator for Padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 8: Load Pre-trained PhoBERT Model for Sequence Classification
model = AutoModelForSequenceClassification.from_pretrained('vinai/phobert-base', num_labels=num_labels)

# Step 9: Set Up Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Step 10: Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Map: 100%|██████████| 3166/3166 [00:00<00:00, 25998.48 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/hoang.minh.an/miniconda3/envs/llm/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# Step 11: Train the Model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.317300,0.238168
2,0.199800,0.237729
3,0.129800,0.249551


TrainOutput(global_step=2145, training_loss=0.20010866374124736, metrics={'train_runtime': 115.2279, 'train_samples_per_second': 297.48, 'train_steps_per_second': 18.615, 'total_flos': 2254750433220096.0, 'train_loss': 0.20010866374124736, 'epoch': 3.0})

In [25]:
# Step 12: Evaluate the Model on the Test Set
eval_results = trainer.evaluate(test_ds)
print("\nEvaluation Results:")
print(eval_results)


Evaluation Results:
{'eval_loss': 0.3011120557785034, 'eval_runtime': 2.5934, 'eval_samples_per_second': 1220.786, 'eval_steps_per_second': 76.347, 'epoch': 3.0}


In [28]:
import torch
# Step 13: Test the Model with Sample Texts
def predict_sentiment(text):
    # Preprocess the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(trainer.model.device) for k, v in inputs.items()}
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    
    # Return the predicted sentiment
    return predicted_class

# Sample texts for testing
sample_texts = [
    "Giáo viên rất nhiệt tình và bài giảng rất hay.",
    "Môi trường học tập chưa tốt, có nhiều vấn đề về trang thiết bị.",
    "Chương trình học rất hữu ích và giúp em hiểu rõ hơn về kiến thức ngành."
]

# Predict and print the sentiment for each sample text
for text in sample_texts:
    sentiment = predict_sentiment(text)
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\n")

Text: Giáo viên rất nhiệt tình và bài giảng rất hay.
Predicted Sentiment: 2

Text: Môi trường học tập chưa tốt, có nhiều vấn đề về trang thiết bị.
Predicted Sentiment: 0

Text: Chương trình học rất hữu ích và giúp em hiểu rõ hơn về kiến thức ngành.
Predicted Sentiment: 2



In [29]:
trainer.model.save_pretrained("phobert-sentiment-analysis")

In [31]:
num_labels

3

{'<s>': 0,
 '<pad>': 1,
 '</s>': 2,
 '<unk>': 3,
 ',': 4,
 '.': 5,
 'và': 6,
 'của': 7,
 'là': 8,
 'các': 9,
 'có': 10,
 'được': 11,
 'trong': 12,
 'cho': 13,
 'đã': 14,
 'với': 15,
 'một': 16,
 'không': 17,
 'người': 18,
 ')': 19,
 '(': 20,
 'những': 21,
 '"': 22,
 'này': 23,
 'để': 24,
 'ở': 25,
 'khi': 26,
 ':': 27,
 'về': 28,
 'năm': 29,
 'đến': 30,
 '-': 31,
 'cũng': 32,
 'vào': 33,
 'trên': 34,
 'tại': 35,
 'nhiều': 36,
 'đó': 37,
 'sẽ': 38,
 'từ': 39,
 'ra': 40,
 'phải': 41,
 'như': 42,
 'ngày': 43,
 'lại': 44,
 'bị': 45,
 'ông': 46,
 'làm': 47,
 'hơn': 48,
 'việc': 49,
 'còn': 50,
 'nhưng': 51,
 'đang': 52,
 'sau': 53,
 'thì': 54,
 'biết': 55,
 'Việt_Nam': 56,
 'đi': 57,
 'nước': 58,
 'rất': 59,
 'mới': 60,
 'sự': 61,
 'có_thể': 62,
 'theo': 63,
 'mà': 64,
 ';': 65,
 'chỉ': 66,
 'nhất': 67,
 'mình': 68,
 'nhà': 69,
 'tôi': 70,
 'trước': 71,
 'lên': 72,
 'con': 73,
 'vẫn': 74,
 'tới': 75,
 '2': 76,
 'nên': 77,
 'tháng': 78,
 'Theo': 79,
 'đồng': 80,
 'cùng': 81,
 'hai': 82,
 'an